# Capture The Flag Shitty Add-On

A (partial so far) solution by [Blake Burkhart](https://bburky.com)

## CTF Details

This IPython notebook is a solution for the [Capture The Flag Shitty Add-On](https://blog.wokwi.com/capture-the-flag-shitty-add-on/) by Uri Shaked. Read the blog post for full details of the CTF challenge.

The following resources will come in handy:

* [Blog post and CTF description](https://blog.wokwi.com/capture-the-flag-shitty-add-on/)
* [`ctf-firmware.ino` source for the CTF](https://github.com/urish/ctf-shittyaddon/blob/master/ctf-firmware/ctf-firmware.ino)
* [ATtiny8x Datasheet](https://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-2586-AVR-8-bit-Microcontroller-ATtiny25-ATtiny45-ATtiny85_Datasheet.pdf)

## Setup

I tested this solution with a Raspberry Pi 2 directly connected to a Digispark clone (an ATtiny85 board). The Digispark was powered from the Pi's 3.3v pin. The [`ctf-firmware-1.0.0.hex`](https://github.com/urish/ctf-shittyaddon/releases/tag/1.0.0) file was flashed to the Digispark.

The Pi must have I²C and SPI enabled in `/boot/config.txt`:

    dtparam=i2c_arm=on,i2c_arm_baudrate=10000
    dtparam=spi=on
  
I set the I²C baud rate to 10kHz to avoid errors. I think this may be due the fact that Raspberry Pis [don't support clock stretching](http://www.advamation.com/knowhow/raspberrypi/rpi-i2c-bug.html) but the [attiny85 and the TinyWire library may allow it](https://github.com/lucullusTheOnly/TinyWire/issues/1). In other words, the slow ATtiny85 may not send all the data fast enough with the Pi's default 100kHz baud rate, set it to something slow like 10kHz. 40kHz was working, but then it quit working some reason.

Also if the AVR ISP is connected to the Pi's SPI pins, the Digispark can be flashed over directly from the Pi with `avrdude`'s linuxspi protocol:

    avrdude -v -pattiny85 -clinuxspi -b14400 -P /dev/spidev0.0 -U flash:w:ctf-firmware-1.0.0.hex 
    avrdude -v -pattiny85 -clinuxspi -b14400 -P /dev/spidev0.0 -Uefuse:w:0xfe:m -Uhfuse:w:0xdf:m -Ulfuse:w:0xe2:m
    
Don't connect the Pi to the Digispark's I²C and SPI at the same time, they share pins and things will go wrong.

# I²C read/write

On an I²C read two bytes are returned: the lower byte of the 16-bit target address, and the byte at the adddress.

When writing, the first byte is used to set the target address. The upper byte is always set to zero. The remainder of the bytes are written in to memory sequentially at the target pointer.

A write with zero data bytes may be performed before a read to set the target address of the read.

Define a pair of `i2c_write()` and `i2c_read()` functions. These functions also track the full 16-byte target address in the `current_target` variable.

In [1]:
from smbus2 import SMBus, i2c_msg
from io import BytesIO

CTF_I2C_ADDR = 0x23

current_target = None

def i2c_write(target, data=[]):
    global current_target

    with SMBus(1) as bus:
        msg = i2c_msg.write(CTF_I2C_ADDR, [target, *data])
        bus.i2c_rdwr(msg)
    current_target = target + len(data)

def i2c_read(target=None, count=1, ):
    global current_target

    with BytesIO() as f:
        with SMBus(1) as bus:
            # If provided, change target by writing the target address, but no data bytes
            if target is not None:
                msg = i2c_msg.write(CTF_I2C_ADDR, [target])
                bus.i2c_rdwr(msg)
                current_target = target

            for i in range(count):
                msg = i2c_msg.read(CTF_I2C_ADDR, 2)
                bus.i2c_rdwr(msg)
                target, data = msg
                f.write(bytes([data]))
                
                if current_target is not None:
                    assert(target == current_target & 0xff)
                current_target += 1
    
        f.seek(0)
        return f.read()

Some quick testing. Manually set the target to 0x00 and read one byte:

In [2]:
i2c_write(0x00)
i2c_read()

b'"'

Read 4 bytes starting at 0x00:

In [3]:
i2c_read(target=0x00, count=4)

b'"\x00\x04@'

Write 6 bytes starting at 0x00:

In [4]:
# i2c_write(0x00, b'FOOBAR')

# Set LED via I2C

In [5]:
# Values appear in datasheets, I found __SFR_OFFSET in avr-libc
SFR_OFFSET = 0x20
PORTB_ADDR = 0x18 + SFR_OFFSET
DDRB_ADDR = 0x17 + SFR_OFFSET

In [6]:
# Read the current value of DDRB
ddrb, = i2c_read(DDRB_ADDR)

# Configure LED as an output pin
# Enable DDB1 for the LED on a digispark 
# I think this is the same pin on the shittyaddon? It uses "pin 6 of the ATtiny (a.k.a PB1)"
ddrb |= 1 << 1

# Set DDRB on the board
i2c_write(DDRB_ADDR, [ddrb])

# Set all output bits in PORTB to turn on the LED
i2c_write(PORTB_ADDR, [0xff])

# The actual CTF board is active-high unlike the Digispark
# To turn the LED on on it set PORTB to 0x00
# i2c_write(PORTB_ADDR, [0x00])

## Dump flash and find the flag

Dump the entire flash (plus some, it loops if you read too much?) and search for the flag with a regex on the bytes.

In [7]:
dump = i2c_read(target=0x00, count=0x10000)

In [8]:
import re

flag_match = re.search(b'\$FLAG:.*?(?=\x00)', dump)
flag = flag_match.group(0)
print(flag.decode('ascii'))

flag_address = flag_match.start()
print('Flag address:', hex(flag_address))

$FLAG:you_found_it!!! :-)
Flag address: 0x801e


## Read from high memory faster

It's annoying that we can't set the `target` to a full two byte `uint16_t` value and address all of memory. However, it's possible to find the upper byte of the flag pointer in memory and set it directly. If we set it to 0x7f and read 0x100 bytes, we should be able to read from higher memory addresses directly.

I compiled the CTF source code using [PlatformIO](https://platformio.org/) with debugging symbols enabled, and took the compiled elf and opened it with the [Ghidra](https://ghidra-sre.org/) dissasembler. After confirming that my recompilation was similar enough, I found the `target` pointer at address `mem:0064` in Ghidra by filtering the symbol list to some interesting addresses:

![Symbol table](symbols.png)

These addresses can also be found with `readelf`, but it's not obvious each variable is a code or data address. You have to somehow figure out that memory is at 0x800000 and code starts at zero.

```
$ readelf -s firmware.elf | grep -E 'flag|flash|target'
    21: 00800064     2 OBJECT  LOCAL  DEFAULT    3 target
    40: 00800060     2 OBJECT  LOCAL  DEFAULT    2 flash_value
    41: 0000001e    13 OBJECT  LOCAL  DEFAULT    1 _ZL4flag
    42: 00800062     2 OBJECT  LOCAL  DEFAULT    2 flash_addr
```

As a first test I tried writing a byte to address 0x64 and as expected the assert inside `i2c_read()` triggered, because the target pointer address was changed in memory. If we update `current_target` we can see it is possible to change the `target` pointer manually:

In [9]:
i2c_write(0x64, [0x00])
current_target = 0x00
i2c_read()

b'"'

Now, set the high byte to read from higher memory addresses. A probem is that if we set address 0x65 to 0x80 we will then start reading at 0x8066. We won't be able to control the lower byte. If we decrease the high byte we can just read more data unill we get to the desired lower byte range.

In [10]:
i2c_write(0x65, [0x7f])
current_target = 0x7f66
dump2 = i2c_read(count=0x100)

match = re.search(b'\$FLAG:.*?(?=\x00)', dump2)
print(f'Flag address {hex(flag_address)} == {hex(match.start() + 0x7f66)}')

Flag address 0x801e == 0x801e


Define a new `i2c_read_high()` function to implement this new read logic precicely:

In [11]:
TARGET_HI_ADDR = 0x65

def i2c_read_high(target, count=1):
    global current_target
    
    target_hi = target >> 8
    # Deliberately undershoot the target address by decrementing target_hi by one
    target_hi -= 1
    i2c_write(TARGET_HI_ADDR, [target_hi])
    current_target = (target_hi << 8) | (TARGET_HI_ADDR + 1)
    # Read and discard the returned bytes until we get to the desired target address
    i2c_read(count=target-current_target)
    
    return i2c_read(count=count)

# A quick test
i2c_read_high(flag_address, count=len(flag))

b'$FLAG:you_found_it!!! :-)'

# Blinking rootkit

While there may be other ways to make the LED blink, I chose to configure Timer/Counter1 to output to PB1.

The datasheet describes how the OC1A output from Timer/Counter1 can be sent to PB1:

> **Port B, Bit 1 – MISO/AIN1/OC0B/OC1A/DO/PCINT1**
> * OC1A: Output Compare Match output: The PB1 pin can serve as an external output for the Timer/Counter1 Compare Match B when configured as an output (DDB1 set). The OC1A pin is also the output pin for the PWM mode timer function.

Read the datasheet's register description for TCCR1 for full details and then configure the TCCR1 timer. The LED will blink indefinately without further input.

In [12]:
TCCR1_ADDR = 0x30 + SFR_OFFSET

# Ensure DDRB is still configured to output PB1
ddrb, = i2c_read(DDRB_ADDR)
ddrb |= 0b00000010
i2c_write(DDRB_ADDR, [ddrb])

# Configure TCCR1:
#   Set bit 7 clear timer on compare match
#   Set bit 6 to 0 to leave PWM disabled
#   Set bits 5:4 to 01 to toggle PB1/OC1A (the LED)
#   Set bits 3:0 to 1111 for prescaler/16384 (as slow as possible)
tccr1 = 0b1_0_01_1111
i2c_write(TCCR1_ADDR, [tccr1])


# TODO: Replace the flag

These will require finally figuring out whatever is acually going on in `loop()`.